# Emission lines

In addition to creating and manipulating spectral energy distributions, `synthesizer` can also create `Line` objects, or more usefully collections of emission lines, `LineCollection` objects, that can be further analysed or manipulated. 

Like spectral energy distributions lines can be extracted directly from `Grid` objects or generated by `Galaxy` objects.

## Extracting lines from `Grid` objects

Grids that have been post-processed through CLOUDY also contain information on nebular emission lines. These can be loaded like regular grids, but there are a number of additional methods for working with lines as demonstrated in these examples:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from synthesizer.grid import Grid

Let's begin by initialising a grid:

In [ ]:
grid_dir = "../../tests/test_grid"
grid_name = "test_grid"
grid = Grid(grid_name, grid_dir=grid_dir)

We can easily get a list of the available lines:

In [ ]:
print(grid.available_lines)

This is also reported if we give the grid to the `print` function:

In [ ]:
print(grid)

Let's choose an age and metallicity we want to get predictions. The in-built method will find the nearest grid point:

In [ ]:
log10age = 6.0  # log10(age/yr)
metallicity = 0.01
# find nearest grid point
grid_point = grid.get_grid_point((log10age, metallicity))

Let's get information on a single line, in this case H-beta:

In [ ]:
line_id = "H 1 4862.69A"
line = grid.get_line(grid_point, "H 1 4862.69A")
print(line)

We can do this for a combination of lines (e.g. a doublet) like this. Note: this sums the contribution of each line. If you want separate lines using the `get_lines` method described below.

In [ ]:
line = grid.get_line(
    grid_point, ["H 1 4862.69A", "O 3 4958.91A", "O 3 5006.84A"]
)
print(line)

We can also create a `LineCollection` a collection of lines which have methods for calculating ratios and diagrams. By default this will create a collection for all available lines but you can also specify which lines you want.

In [ ]:
lines = grid.get_lines(grid_point)
print(lines)

We can measure some predifined line ratios:

In [ ]:
ratio_id = "BalmerDecrement"
ratio = lines.get_ratio(ratio_id)
print(f"{ratio_id}: {ratio:.2f}")

Or loop over all pre-defined ratios:

In [ ]:
for ratio_id in lines.available_ratios:
    ratio = lines.get_ratio(ratio_id)
    print(f"{ratio_id}: {ratio:.2f}")

We can plot a ratio against metallicity by looping over the metallicity grid:

In [ ]:
ratio_id = "R23"
ia = 0  # 1 Myr old for test grid
ratios = []
for iZ, Z in enumerate(grid.metallicity):
    grid_point = (ia, iZ)
    lines = grid.get_lines(grid_point)
    ratios.append(lines.get_ratio(ratio_id))

Zsun = grid.metallicity / 0.0124
plt.plot(Zsun, ratios)
plt.xlim([0.01, 1])
plt.ylim([1, 20])
plt.xscale("log")
plt.yscale("log")
plt.xlabel(r"$Z/Z_{\odot}$")
plt.ylabel(lines.get_ratio_label(ratio_id))
plt.show()

We can also generate "diagrams" pairs of line ratios like the BPT diagram

In [ ]:
diagram_id = "BPT-NII"
ia = 0  # 1 Myr old for test grid
x = []
y = []
for iZ, Z in enumerate(grid.metallicity):
    grid_point = (ia, iZ)
    lines = grid.get_lines(grid_point)
    x_, y_ = lines.get_diagram(diagram_id)
    x.append(x_)
    y.append(y_)

plt.plot(x, y)
plt.xlim([0.01, 10])
plt.ylim([0.05, 20])
plt.xscale("log")
plt.yscale("log")

# grab x and y labels, this time use "fancy" label ids
xlabel, ylabel = lines.get_diagram_label(diagram_id, fancy=True)

plt.xlabel(xlabel)
plt.ylabel(ylabel)
plt.show()

## Lines from `Galaxy` objects